In [49]:
import warnings

from tqdm import tqdm
import pandas as pd
import numpy as np
import weighted

from jre_utils.datapath import DATA_DIRECTORY_PATH
from jp_prefecture.address import JpAddressParser

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [50]:
"""
Goal: 
For the PLPS data, 
index by year, prefecture and municipality, then
get the weighted average price per unit area
DONE!!
"""

plps_data_path = f"{DATA_DIRECTORY_PATH}/core_scraped/plps"
plps_data_paths = {
    year: f"{plps_data_path}/{year}.csv"
    for year in range(1997, 2024)
}
print(plps_data_paths[2022])

plps_pre_derived_data_path = f"{DATA_DIRECTORY_PATH}/pre_derived_plps"
plps_pre_derived_data_paths = {
    year: f"{plps_pre_derived_data_path}/yearly/{year}.csv"
    for year in range(1997, 2024)
}
print(plps_pre_derived_data_paths[2022])

../../data/core_scraped/plps/2022.csv
../../data/pre_derived_plps/yearly/2022.csv


In [51]:
# convert address to city code
addr_column = "所在及び地番"
price_per_unit_area_column = "価格(円/m²)"
area_column = "地積(m²)"

required_columns = ["year", "area_code", "area", "unit_price", "traded_area", "count"]


In [52]:
import json
from pprint import pprint

sub_city_to_city_path = f"{DATA_DIRECTORY_PATH}/core_scraped/sub_city_to_city.json"
with open(sub_city_to_city_path) as fd:
     sub_city_to_city = json.load(fd)
     pprint(f"E.g. Maps 1101 to {sub_city_to_city["1101"]}")

area_code_to_area_path = f"{DATA_DIRECTORY_PATH}/core_scraped/area_code_to_area.json"
with open(area_code_to_area_path) as fd:
     area_code_to_area = json.load(fd)
     pprint(f"E.g. Maps 1100 to {area_code_to_area["1100"]}") 

def get_area_from_area_code(area_code):
     return area_code_to_area.get(area_code, "na" )

'E.g. Maps 1101 to 1100'
'E.g. Maps 1100 to Hokkaido Sapporo-shi'


In [53]:
def prepare_area_column(area):
    try:
        return int(area.split("(")[0].replace(",", ""))
    except:
        print("error")
        return np.NaN

def get_city_code(parser, addr):
    try:
        city_code = str(parser.parse_address(addr).cityCode)
        return sub_city_to_city.get(city_code, city_code)
    except:
        return addr

def get_area_from_area_code(area_code):
     return area_code_to_area.get(area_code, "na" )

parser = JpAddressParser(enable_town=True)

In [54]:
def custom_aggregate(x):
    d = {}
    d["unit_price_wmean"] = np.average(x["unit_price"], weights=x["traded_area"])
    d["unit_price_wmedian"] = weighted.median(x["unit_price"], weights=x["traded_area"])
    d["unit_price_mean"] = x["unit_price"].mean()
    d["unit_price_median"] = x["unit_price"].median()
    d["total_traded_area"] = x["traded_area"].sum()
    d["count"] = x["count"].count()
    return pd.Series(
        d,
        index=[
            "unit_price_wmean",
            "unit_price_wmedian",
            "unit_price_mean",
            "unit_price_median",
            "total_traded_area",
            "count",
        ],
    )

In [56]:
main_df = pd.read_csv(plps_data_paths[2022])

In [57]:
main_df

,基準地番号,調査基準日,所在及び地番,住居表示,価格(円/m²),交通施設、距離,地積(m²),形状（間口：奥行き）,利用区分、構造,利用現況,給排水等状況,周辺の土地の利用現況,前面道路の状況,その他の接面道路,用途区分、高度地区、防火・準防火,建ぺい率（%）、容積率（%）,都市計画区域区分,森林法、公園法、 自然環境等
0,札幌中央-1,令和4年7月1日,北海道札幌市中央区宮の森３条５丁目８番１ 地図で確認する,宮の森３条５−１−３１,"235,000(円/m²)",地下鉄西２８丁目、 650m,237(m²),(1.0:1.2),建物などの敷地、W（木造） 2F,住宅,ガス ・ 水道 ・ 下水,一般住宅、アパート、マンション等が建ち並ぶ住宅地域,北東 10.0m 市区町村道,NaN,第一種中高層住居専用地域,60(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t200(%),市街化区域,NaN
1,札幌中央-2,令和4年7月1日,北海道札幌市中央区南１８条西８丁目５９３番１８外 地図で確認する,南１８条西８−１−４３,"200,000(円/m²)",地下鉄幌平橋、 800m,146(m²),(1.0:2.0),建物などの敷地、W（木造） 2F B1,住宅,ガス ・ 水道 ・ 下水,一般住宅、マンション等が建ち並ぶ既成住宅地域,北 10.9m 市区町村道,NaN,第一種住居地域,60(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t200(%),市街化区域,NaN
2,札幌中央-3,令和4年7月1日,北海道札幌市中央区南２２条西８丁目６３０番４外 地図で確認する,南２２条西８−３−２６,"170,000(円/m²)","地下鉄幌平橋、 1,600m",233(m²),(1.0:1.5),建物などの敷地、W（木造） 2F B1,住宅,ガス ・ 水道 ・ 下水,一般住宅、アパート、マンション等が建ち並ぶ住宅地域,西 12.7m 市区町村道,NaN,第二種中高層住居専用地域,60(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t200(%),市街化区域,NaN
3,札幌中央-4,令和4年7月1日,北海道札幌市中央区界川２丁目１８８０番４１０ 地図で確認する,界川２−４−２４,"57,000(円/m²)","地下鉄円山公園、 2,500m",279(m²),(1.0:1.5),建物などの敷地、W（木造） 2F B1,住宅,ガス ・ 水道 ・ 下水,中規模一般住宅が建ち並ぶ傾斜地の住宅地域,南 8.0m 市区町村道,NaN,第一種低層住居専用地域,40(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t80(%),市街化区域,NaN
4,札幌中央-5,令和4年7月1日,北海道札幌市中央区宮ヶ丘２丁目４７４番８６ 地図で確認する,宮ヶ丘２−１−２０,"355,000(円/m²)",地下鉄西２８丁目、 500m,370(m²),(1.0:2.0),建物などの敷地、W（木造） 3F,住宅,ガス ・ 水道 ・ 下水,中規模一般住宅の中にマンションも見られる閑静な住宅地域,西 7.2m 市区町村道,NaN,第一種中高層住居専用地域,60(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t200(%),市街化区域,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20993,竹富-2,令和4年7月1日,沖縄県八重山郡竹富町字小浜村内７７番 地図で確認する,NaN,"5,200(円/m²)","小浜港、 1,500m",593(m²),(1.0:1.5),建物などの敷地、W（木造） 1F,住宅,ガス ・ 水道 ・ 下水,農家住宅の多く建ち並ぶ農家集落地域,南西 5.0m 都道府県道,三方路,NaN,-(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t-(%),都市計画区域の定めのない区域,NaN
20994,竹富-3,令和4年7月1日,沖縄県八重山郡竹富町字南風見仲屋敷２９番３ 地図で確認する,NaN,"9,500(円/m²)",大富停、 230m,708(m²),(1.2:1.0),建物などの敷地、RC（鉄筋 コンクリート造） 2F,住宅,ガス ・ 水道 ・ 下水,農家住宅のほか空地も介在する区画整然とした農家集落地域,南 4.5m 市区町村道,三方路,NaN,-(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t-(%),都市計画区域の定めのない区域,NaN
20995,竹富-4,令和4年7月1日,沖縄県八重山郡竹富町字竹富東屋敷４５９番 地図で確認する,NaN,"5,800(円/m²)","竹富東港、 1,000m",531(m²),(1.0:1.2),建物などの敷地、W（木造） 1F,住宅,ガス ・ 水道 ・ 下水,農家住宅及び民宿等の混在する地域,南 4.0m 市区町村道 未舗装,三方路,NaN,-(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t-(%),都市計画区域の定めのない区域,NaN
20996,与那国-1,令和4年7月1日,沖縄県八重山郡与那国町字与那国祖納１９番 地図で確認する,NaN,"11,600(円/m²)",祖納停、 120m,411(m²),(1.0:1.5),建物などの敷地、RC（鉄筋 コンクリート造） 1F,住宅,ガス ・ 水道 ・ 下水,一般住宅、飲食店舗等が混在する地域,南 12.0m 都道府県道,背面道,NaN,-(%)\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t-(%),都市計画区域の定めのない区域,NaN


In [23]:
# %%capture

# for year, data_path in plps_data_paths.items():
    # main_df = pd.read_csv(data_path)
    main_df = main_df.assign(year=year, count=1)
    main_df["area_code"] = main_df[addr_column].apply(lambda x: get_city_code(parser, x))
    main_df["area"] = main_df["area_code"].apply(get_area_from_area_code)
    main_df["unit_price"] = main_df[price_per_unit_area_column].apply(prepare_area_column)
    main_df["traded_area"] = main_df[area_column].apply(prepare_area_column)
    main_df = main_df[main_df["area_code"].apply(lambda x: x.isdigit())]
    main_df.to_csv(f"{DATA_DIRECTORY_PATH}/pre_derived_plps/yearly/{year}.csv", index=False)

../../data/core_scraped/plps/1997.csv
(27426, 18)
../../data/core_scraped/plps/1998.csv
(27616, 18)
../../data/core_scraped/plps/1999.csv
(25565, 18)
../../data/core_scraped/plps/2000.csv
(25220, 18)
../../data/core_scraped/plps/2001.csv
(25238, 18)
../../data/core_scraped/plps/2002.csv
(25238, 18)
../../data/core_scraped/plps/2003.csv
(25246, 18)
../../data/core_scraped/plps/2004.csv
(25165, 18)
../../data/core_scraped/plps/2005.csv
(24301, 18)
../../data/core_scraped/plps/2006.csv
(23294, 18)
../../data/core_scraped/plps/2007.csv
(22455, 18)
../../data/core_scraped/plps/2008.csv
(21911, 18)
../../data/core_scraped/plps/2009.csv
(21281, 18)
../../data/core_scraped/plps/2010.csv
(20993, 18)
../../data/core_scraped/plps/2011.csv
(20743, 18)
../../data/core_scraped/plps/2012.csv
(20539, 18)
../../data/core_scraped/plps/2013.csv
(21421, 18)
../../data/core_scraped/plps/2014.csv
(21201, 18)
../../data/core_scraped/plps/2015.csv
(21194, 18)
../../data/core_scraped/plps/2016.csv
(21138, 18)


In [40]:
final_df = pd.DataFrame()

pbar = tqdm(total=len(plps_pre_derived_data_paths))

for year, data_path in plps_pre_derived_data_paths.items():
    main_df = pd.read_csv(data_path)
    df = main_df[required_columns]
    df = (
        df.groupby(["year", "area_code", "area"])
        .apply(custom_aggregate)
        .reset_index()
    )
    df = df.sort_values(by=["year", "area_code"], ascending=[False, True]).reset_index(drop=True)
    df.to_csv(f"{DATA_DIRECTORY_PATH}/derived_plps/yearly/{year}.csv", index=False)
    final_df = pd.concat([final_df, df])
    
    pbar.update()
pbar.close()
    

100%|██████████| 27/27 [00:14<00:00,  1.88it/s]


In [44]:
final_df.to_csv(f"{DATA_DIRECTORY_PATH}/derived_plps/combined.csv", index=False)

In [48]:
final_df[final_df["area_code"] == 13102]

,year,area_code,area,unit_price_wmean,unit_price_wmedian,unit_price_mean,unit_price_median,total_traded_area,count
538,1997,13102,Tokyo-to Chuo-ku,3.711818e+06,2.261302e+06,3.180833e+06,2120000.0,14887.0,42.0
538,1998,13102,Tokyo-to Chuo-ku,3.617594e+06,2.058182e+06,3.063095e+06,1910000.0,14872.0,42.0
539,1999,13102,Tokyo-to Chuo-ku,3.844427e+06,2.114391e+06,2.999970e+06,1830000.0,10413.0,33.0
539,2000,13102,Tokyo-to Chuo-ku,3.640613e+06,1.825487e+06,2.845806e+06,1500000.0,10115.0,31.0
543,2001,13102,Tokyo-to Chuo-ku,3.485010e+06,1.648035e+06,2.700968e+06,1380000.0,10115.0,31.0
545,2002,13102,Tokyo-to Chuo-ku,3.406168e+06,1.514619e+06,2.613226e+06,1250000.0,10128.0,31.0
547,2003,13102,Tokyo-to Chuo-ku,3.405375e+06,1.465027e+06,2.592387e+06,1200000.0,10170.0,31.0
547,2004,13102,Tokyo-to Chuo-ku,3.477969e+06,1.453239e+06,2.645452e+06,1170000.0,10170.0,31.0
568,2005,13102,Tokyo-to Chuo-ku,3.750174e+06,1.502301e+06,2.875500e+06,1220000.0,10032.0,30.0
613,2006,13102,Tokyo-to Chuo-ku,4.346117e+06,1.582655e+06,3.388967e+06,1355000.0,10216.0,30.0
